# Converting Original Format into BIDs format
1. Set the new BIDs directory name
2. Load up the .set files from the original folder
3. Set the digital montage
4. Load up the events
5. Write the BID files


In [1]:
import os.path as op
import shutil
import pandas as pd
from mne import channels
import mne
from mne.datasets import eegbci

from mne_bids import write_raw_bids, BIDSPath, print_dir_tree
from mne_bids.stats import count_events

In [ ]:
ppt_num='3'
session = '3'

### New BIDs directory name

In [12]:
output_path = 'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\BIDS\\'

### Create digital montage

In [7]:
# montage
montage_path = 'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\montages\\'
df = pd.read_csv(montage_path+'dig.txt',sep="\t")
print(df.keys())
ch_names = df['name'].to_list()

pos = df[['x','y','z']].values
dig_ch_pos = dict(zip(ch_names,pos))
montage = mne.channels.make_dig_montage(ch_pos=dig_ch_pos)#,coord_frame = 'head')

montage.plot(kind='topomap', show_names=True)

Index(['x', 'y', 'z', 'name'], dtype='object')


### Load up raw data and set montage

In [9]:
pathdata = 'sub'+ppt_num+'_sess'+session+ '\\sub'+ppt_num+'_sess'+session+'.set' 
xpath = 'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\dataset\\'
raw_path = xpath + pathdata
eog =  ['VEOGL', 'VEOGU', 'HEOGR', 'HEOGL'] # electroculogram electrodes 
raw = mne.io.read_raw_eeglab(raw_path, preload=False, eog=eog)
raw.info['line_freq'] = 50

raw.set_montage(montage)
#print(raw.info['dig'])
#raw.info['dig'] ='None'

#raw.set_montage(montage)
#ant_montage.plot(kind='topomap', show_names=True)

Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\dataset\sub3_sess3\sub3_sess3.fdt
[<DigPoint |     EEG #1 : (88557.0, 30881.0, 55218.0) mm : head frame>, <DigPoint |     EEG #2 : (99177.0, 7755.0, 52688.0) mm : head frame>, <DigPoint |     EEG #3 : (97849.0, -26865.0, 55500.0) mm : head frame>, <DigPoint |     EEG #4 : (42725.0, 70823.0, 60565.0) mm : head frame>, <DigPoint |     EEG #5 : (52553.0, 46751.0, 102230.0) mm : head frame>, <DigPoint |     EEG #6 : (70176.0, 393.0, 113792.0) mm : head frame>, <DigPoint |     EEG #7 : (63513.0, -44865.0, 97765.0) mm : head frame>, <DigPoint |     EEG #8 : (59113.0, -71507.0, 55565.0) mm : head frame>, <DigPoint |     EEG #9 : (18170.0, 71737.0, 90319.0) mm : head frame>, <DigPoint |    EEG #10 : (34162.0, 25469.0, 131336.0) mm : head frame>, <DigPoint |    EEG #11 : (42685.0, -33885.0, 125432.0) mm : head frame>, <DigPoint |    EEG #12 : (37058.0, -74517.0, 78369.0) mm : head frame>, <DigPoint |    EEG #13 : (-29352.0, 69687.0, 7196

C:\Users\hlw69\AppData\Local\Temp\ipykernel_11000\1659058453.py:7: RuntimeWarning: Not setting positions of 4 eog channels found in montage:
['HEOGR', 'HEOGL', 'VEOGU', 'VEOGL']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_eeglab(raw_path, preload=False, eog=eog)
C:\Users\hlw69\AppData\Local\Temp\ipykernel_11000\1659058453.py:10: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


### Get events

In [10]:
# get events
events, event_id = mne.events_from_annotations(raw, verbose = False)
events_data = xpath+'sub'+ppt_num+'_sess'+session+ '\\sub'+ppt_num+'_sess'+session+'.fdt'

### Write BIDs!!!

In [15]:
bids_path = BIDSPath(subject='0'+ppt_num, session='0'+session,
                     task='experiment', run='01', root=output_path)
write_raw_bids(raw, bids_path, 
               event_id=event_id, overwrite=True, montage= montage)

Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\dataset\sub3_sess3\sub3_sess3.fdt
Writing 'X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\BIDS\participants.tsv'...
Writing 'X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\BIDS\participants.json'...
Used Annotations descriptions: ['0, Imagination_a_flower_high_10###my_stream_name', '0, Imagination_a_flower_high_11###my_stream_name', '0, Imagination_a_flower_high_17###my_stream_name', '0, Imagination_a_flower_high_23###my_stream_name', '0, Imagination_a_flower_high_4###my_stream_name', '0, Imagination_a_flower_high_5###my_stream_name', '0, Imagination_a_flower_low_10###my_stream_name', '0, Imagination_a_flower_low_13###my_stream_name', '0, Imagination_a_flower_low_17###my_stream_name', '0, Imagination_a_flower_low_23###my_stream_name', '0, Imagination_a_flower_low_25###my_stream_name', '0, Imagination_a_flower_low_5###my_stream_name', '0, Imagination_a_flower_normal_10###my_stream_name', '0, Imagination_a_flower_normal_11##

Writing 'X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\BIDS\sub-03\ses-03\eeg\sub-03_ses-03_task-experiment_run-01_events.tsv'...
Writing 'X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\BIDS\dataset_description.json'...
Writing 'X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\BIDS\sub-03\ses-03\eeg\sub-03_ses-03_task-experiment_run-01_eeg.json'...
Writing 'X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\BIDS\sub-03\ses-03\eeg\sub-03_ses-03_task-experiment_run-01_channels.tsv'...
Copying data files to sub-03_ses-03_task-experiment_run-01_eeg.set


C:\Users\hlw69\AppData\Local\Temp\ipykernel_11000\1965997066.py:3: RuntimeWarning: Not setting positions of 4 eog channels found in montage:
['HEOGR', 'HEOGL', 'VEOGU', 'VEOGL']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  write_raw_bids(raw, bids_path,
C:\Users\hlw69\AppData\Local\Temp\ipykernel_11000\1965997066.py:3: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(raw, bids_path,


Writing 'X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\BIDS\sub-03\ses-03\sub-03_ses-03_scans.tsv'...
Wrote X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\BIDS\sub-03\ses-03\sub-03_ses-03_scans.tsv entry with eeg\sub-03_ses-03_task-experiment_run-01_eeg.set.


BIDSPath(
root: X:/CompSci/ResearchProjects/EJONeill/Neuroimaging/BIDS
datatype: eeg
basename: sub-03_ses-03_task-experiment_run-01_eeg.set)